In [125]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor

In [134]:
df_train = pd.read_excel('data/Data_Train.xlsx')

In [135]:
df_train.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625


In [136]:
df_train['Destination'].replace({'New Delhi' : 'Delhi'}, inplace = True)

In [137]:
df_train.Total_Stops.fillna('non-stop', inplace = True)

In [138]:
df_train['Total_Stops'].replace({'non-stop' : '0 stop'}, inplace = True)

In [139]:
df_train.Route.fillna('BLR → DEL', inplace = True)

In [140]:
df_train['Additional_Info'].replace({'No info' : 'No Info'}, inplace = True)

In [141]:
from sklearn.preprocessing import LabelEncoder

In [142]:
la = LabelEncoder()
la.fit(df_train.Airline)
df_train.Airline = la.transform(df_train.Airline)

In [143]:
lc = LabelEncoder()
lc.fit(['Banglore', 'Chennai', 'Delhi', 'Kolkata', 'Cochin', 'Hyderabad', 'Mumbai'])
df_train.Source = lc.transform(df_train.Source)
df_train.Destination = lc.transform(df_train.Destination)

In [144]:
lts = LabelEncoder()
lts.fit(['0 stop', '1 stop', '2 stops', '3 stops', '4 stops', '5 stops'])
df_train.Total_Stops = lts.transform(df_train.Total_Stops)

In [145]:
lai = LabelEncoder()
lai.fit(df_train.Additional_Info)
df_train.Additional_Info = lai.transform(df_train.Additional_Info)

In [146]:
lts.classes_

array(['0 stop', '1 stop', '2 stops', '3 stops', '4 stops', '5 stops'],
      dtype='<U7')

In [147]:
df_train.Route = df_train.Route.str.split(' → ')

In [148]:
df_train.Route[0][0]

'BLR'

In [149]:
city_code = set()
for i in range(len(df_train)):
    del df_train.Route[i][0]
    del df_train.Route[i][-1]
    for j in range(len(df_train.Route[i])):
        city_code.add(df_train.Route[i][j])

In [150]:
len(city_code)

42

In [151]:
df_train['Duration'].replace({'5m' : '0h 5m'}, inplace=True)

In [152]:
hour = df_train.Duration.str.split('h')
for i in range (len(hour)):
    if((hour[i][1]) == ''):
        hour[i][1] = ' 0m'

In [153]:
for i in range (len(hour)):
    hour[i][0] = int(hour[i][0])*60
    hour[i][1] = int(hour[i][1].split('m')[0])
    hour[i][0] = hour[i][0] + hour[i][1]

In [154]:
for i in range (len(hour)):
    hour[i] = hour[i][0]
df_train.Duration = hour.astype(int)

In [155]:
dep = df_train.Dep_Time.str.split(':')
for i in range(len(dep)):
    dep[i] = dep[i][0] + dep[i][1]
df_train.Dep_Time = dep.astype(int)

In [156]:
arr = df_train.Arrival_Time.str.split(' ')
for i in range (len(arr)):
    arr[i] = arr[i][0]
arr = arr.str.split(':')
for i in range(len(arr)):
    arr[i] = arr[i][0] + arr[i][1]
df_train.Arrival_Time = arr.astype(int)

In [157]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)


In [158]:
add_datepart(df_train, "Date_of_Journey")

In [218]:
#df_train_tmp = df_train

In [219]:
#df_train_con = pd.concat([df_train_tmp,pd.DataFrame(columns=list(city_code))], sort=False)

In [220]:
#df_train_con.fillna(value=False, inplace=True)

In [ ]:
#for i in range (len(df_train_con)):
#    for j in range (len(df_train_con.Route[i])):
#        x = df_train_con.Route[i][j]
#        df_train_con[x][i] = True

In [209]:
#df_train_con['IXR'].head()

0    False
1     True
2    False
3    False
4    False
Name: IXR, dtype: object

In [130]:
df_train.head(10)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,...,Date_of_JourneyDay,Date_of_JourneyDayofweek,Date_of_JourneyDayofyear,Date_of_JourneyIs_month_end,Date_of_JourneyIs_month_start,Date_of_JourneyIs_quarter_end,Date_of_JourneyIs_quarter_start,Date_of_JourneyIs_year_end,Date_of_JourneyIs_year_start,Date_of_JourneyElapsed
0,3,0,3,[],2220,110,170,0,6,3897,...,24,6,83,False,False,False,False,False,False,1553385600
1,1,5,0,"[IXR, BBI]",550,1315,445,2,6,7662,...,1,2,121,False,True,False,False,False,False,1556668800
2,4,3,2,"[LKO, BOM]",925,425,1140,2,6,13882,...,9,6,160,False,False,False,False,False,False,1560038400
3,3,5,0,[NAG],1805,2330,325,1,6,6218,...,12,6,132,False,False,False,False,False,False,1557619200
4,3,0,3,[NAG],1650,2135,285,1,6,13302,...,1,4,60,False,True,False,False,False,False,1551398400
5,8,5,0,[],900,1125,145,0,6,3873,...,24,0,175,False,False,False,False,False,False,1561334400
6,4,0,3,[BOM],1855,1025,930,1,5,11087,...,12,1,71,False,False,False,False,False,False,1552348800
7,4,0,3,[BOM],800,505,1265,1,6,22270,...,1,4,60,False,True,False,False,False,False,1551398400
8,4,0,3,[BOM],855,1025,1530,1,5,11087,...,12,1,71,False,False,False,False,False,False,1552348800
9,6,3,2,[BOM],1125,1915,470,1,6,8625,...,27,0,147,False,False,False,False,False,False,1558915200


In [178]:
df_train_con.isnull().sum()

Airline                            0
Source                             0
Destination                        0
Route                              0
Dep_Time                           0
Arrival_Time                       0
Duration                           0
Total_Stops                        0
Additional_Info                    0
Price                              0
Date_of_JourneyYear                0
Date_of_JourneyMonth               0
Date_of_JourneyWeek                0
Date_of_JourneyDay                 0
Date_of_JourneyDayofweek           0
Date_of_JourneyDayofyear           0
Date_of_JourneyIs_month_end        0
Date_of_JourneyIs_month_start      0
Date_of_JourneyIs_quarter_end      0
Date_of_JourneyIs_quarter_start    0
Date_of_JourneyIs_year_end         0
Date_of_JourneyIs_year_start       0
Date_of_JourneyElapsed             0
PNQ                                0
VGA                                0
JLR                                0
MAA                                0
B

In [169]:
df_train.dtypes

Airline                             int64
Source                              int64
Destination                         int64
Route                              object
Dep_Time                            int64
Arrival_Time                        int64
Duration                            int64
Total_Stops                         int64
Additional_Info                     int64
Price                               int64
Date_of_JourneyYear                 int64
Date_of_JourneyMonth                int64
Date_of_JourneyWeek                 int64
Date_of_JourneyDay                  int64
Date_of_JourneyDayofweek            int64
Date_of_JourneyDayofyear            int64
Date_of_JourneyIs_month_end          bool
Date_of_JourneyIs_month_start        bool
Date_of_JourneyIs_quarter_end        bool
Date_of_JourneyIs_quarter_start      bool
Date_of_JourneyIs_year_end           bool
Date_of_JourneyIs_year_start         bool
Date_of_JourneyElapsed              int64
dtype: object